# Data Exploratory

In [116]:
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import plotly.express as px

In [117]:
df = pd.read_csv("nba_player_stats_C_Rami.csv")

df

,Player,Team,Points,Season
0,Luka Dončić,DAL,2370.0,2024
1,Shai Gilgeous-Alexander,OKC,2254.0,2024
2,Giannis Antetokounmpo,MIL,2222.0,2024
3,Jalen Brunson,NYK,2212.0,2024
4,Nikola Jokić,DEN,2085.0,2024
...,...,...,...,...
17808,Yinka Dare,NJN,0.0,1995
17809,Tom Hovasse,ATL,0.0,1995
17810,Rich King,SEA,0.0,1995
17811,Derrick Phelps,SAC,0.0,1995


In [118]:
df.columns

Index(['Player', 'Team', 'Points', 'Season'], dtype='object')

In [119]:
df.dtypes

Player     object
Team       object
Points    float64
Season      int64
dtype: object

In [120]:
df.describe()

,Points,Season
count,17783.000000,17813.000000
mean,440.588933,2010.470387
std,450.001666,8.721962
min,0.000000,1995.000000
25%,80.000000,2003.000000
50%,292.000000,2011.000000
75%,671.000000,2018.000000
max,2832.000000,2024.000000


In [121]:
print("Missing values per columns:")
print(df.isnull().sum())

Missing values per columns:
Player     0
Team      30
Points    30
Season     0
dtype: int64


In [122]:
def get_player_stats(player_name, df):

    # Filter the dataset for the given player
    player_data = df[df['Player'].str.lower() == player_name.lower()]
    
    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return None
    
    # Sort by season
    player_data = player_data.sort_values(by='Season', ascending=True)
    
    # Print the table
    print(player_data)

    #Plotting the data
    plt.figure(figsize=(10, 5))
    sns.lineplot(x=player_data['Season'], y=player_data['Points'], label=player_name)
    plt.xlabel("Season")
    plt.ylabel("Total Points")
    plt.title(f"Points per Season for {player_name}")
    plt.xticks(player_data['Season'].unique())
    plt.grid(True)
    plt.show()

player_name = input("Enter player name: ")
get_player_stats(player_name, df)

No data found for player: Lebron


In [123]:
import plotly.express as px

df = df.dropna(subset=["Season", "Team", "Points"])  # Removes rows with missing hierarchy
df.fillna("Unknown", inplace=True)  # Replace NaN with 'Unknown'
df.replace("", "Unknown", inplace=True)


fig = px.sunburst(
    df,
    path=["Season", "Team"],  # Remove "Points" from path
    values="Points",
    title="Sunburst Chart: Season, Team, and Points"
)


fig.show()

/var/folders/4y/hb_l24vn64d8fllf91pw21d00000gn/T/ipykernel_27196/1184388805.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/4y/hb_l24vn64d8fllf91pw21d00000gn/T/ipykernel_27196/1184388805.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

